<a href="https://colab.research.google.com/github/Songseunggeon/AiModels/blob/main/potHole%ED%83%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 포트홀 탐지 모델 만들기
# 포트홀이란
  1. 포트홀은 도로에 만들어진 크랙의 종류로 깊게 패인 웅덩이를 말한다.
  2. 포트홀을 탐지하는 목적은
    - 자율 주행 자동차의 안전을 위한 회피 운행
    - 도로 보수를 위한 위치 탐색

## 응용 SW를 개발하는 방법
  1. What ? : 위의 정의와 같이 무엇을 할 것인지를 정한다.
  2. Dataset을 준비한다. : 학습을 위한 annotation된 dataSet을 준비한다.
  3. 적합한 Pre-Trained 된 모델을 구한다. : 이미지 관련서는 우리는 YOLO에 집중한다.
  4. Fine-Tuning : 2에서 준비한 dataSet을 이용하여 custom Model을 만든다.
  5. 응용 SW를 시작한다.
  - webAPP : 실습한 TM을 이용한 JS를 기억한다.
  - nativeAPP : Python과 PyQt5 를 이용한 배포가능한 setup.exe 까지 완성한 것을 기억한다.

# YOLO 클로닝

In [ ]:
# 현재 폴더
%pwd
# 위치 이동을 강제로 해야한다면
%cd /content/
%pwd

/content


'/content'

In [ ]:
# gitHub에서 YOLO 가져오기 https://github.com/ultralytics/yolov5
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16575, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 16575 (delta 28), reused 37 (delta 18), pack-reused 16522
Receiving objects: 100% (16575/16575), 15.03 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (11387/11387), done.


# 작업폴더로 이동

In [ ]:
%cd yolov5

/content/yolov5


In [ ]:
# dataSet을 가져올 폴더를 만듬
%mkdir /content/yolov5/pothole
%cd /content/yolov5/pothole

/content/yolov5/pothole


## dataSet을 가져오기

In [ ]:
!curl -L "https://public.roboflow.com/ds/AZZeoZCGOT?key=XFPYdCUU9Z" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%pwd

'/content/yolov5/pothole'

## DataSet의 구성 내용을 확인

In [ ]:
from glob import glob
train_img_list = glob('/content/yolov5/pothole/train/images/*.jpg')
test_img_list = glob('/content/yolov5/pothole/test/images/*.jpg')
valid_img_list = glob('/content/yolov5/pothole/valid/images/*.jpg')
print(len(train_img_list), len(test_img_list), len(valid_img_list))

465 67 133


## 모델 FineTunning 을 시작

## 1. 데이터셋의 위치등을 알려주기 위한 yaml 파일 준비

## 윗줄에서 만든 파이썬

In [ ]:
import yaml
with open('/content/yolov5/pothole/train.txt','w') as f:
    f.write('\n'.join(train_img_list) + '\n')
with open('/content/yolov5/pothole/test.txt','w') as f:
    f.write('\n'.join(test_img_list) + '\n')
with open('/content/yolov5/pothole/val.txt','w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line,cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

### FineTunning중에 모델에게 dataSet의 위치를 알려주기 위한 data.yaml을 만든다.

In [ ]:
%%writetemplate /content/yolov5/pothole/data.yaml

train: pothole/train/images
test: pothole/test/images
val: pothole/valid/images

nc: 1
names: ['pothole']

In [ ]:
%cat /content/yolov5/pothole/data.yaml


train: pothole/train/images
test: pothole/test/images
val: pothole/valid/images

nc: 1
names: ['pothole']


### 만들어진 data.yaml에서 필요한 변수 (nc) 를 가져오기

In [ ]:
with open('/content/yolov5/pothole/data.yaml','r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])
  print (num_classes)

1


### custom_yolov5.yaml을 만들기
**굵은 텍스트**
1. 원래 pre-trained 된 yolov5s.yaml이 갖고 있는 내용을 수정해서 만들어야함.
  - 원본 yolov5s.yaml을 cat으로 프린트해서 복사하여 사용한다.
2. 원래 있던 yaml 내용중 ,nc 부분만 수정해준다.

In [ ]:
%cat /content/yolov5/models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 

## custom_yolov5s.yaml 파일을 만든다.

+ nc : 80 또는 nc : 다른숫자
부분을 앞서 가져온 변수 num_classes로 바꾸어 준다. {num_classes}

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes} # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-1, 3, C3, [256, False]], # 17 (P3/8-small)

    [-1, 1, Conv, [256, 3, 2]],
    [[-1, 14], 1, Concat, [1]], # cat head P4
    [-1, 3, C3, [512, False]], # 20 (P4/16-medium)

    [-1, 1, Conv, [512, 3, 2]],
    [[-1, 10], 1, Concat, [1]], # cat head P5
    [-1, 3, C3, [1024, False]], # 23 (P5/32-large)

    [[17, 20, 23], 1, Detect, [nc, anchors]], # Detect(P3, P4, P5)
  ]

### 최종 만들어진 custom_yolov5s.yaml의 내용중 nc가 수정되었는지 확인
nc = num_classes = 1

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml


# Parameters
nc: 1 # number of classes
depth_multiple: 0.33 # model depth multiple
width_multiple: 0.50 # layer channel multiple
anchors:
  - [10, 13, 16, 30, 33, 23] # P3/8
  - [30, 61, 62, 45, 59, 119] # P4/16
  - [116, 90, 156, 198, 373, 326] # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [
    [-1, 1, Conv, [64, 6, 2, 2]], # 0-P1/2
    [-1, 1, Conv, [128, 3, 2]], # 1-P2/4
    [-1, 3, C3, [128]],
    [-1, 1, Conv, [256, 3, 2]], # 3-P3/8
    [-1, 6, C3, [256]],
    [-1, 1, Conv, [512, 3, 2]], # 5-P4/16
    [-1, 9, C3, [512]],
    [-1, 1, Conv, [1024, 3, 2]], # 7-P5/32
    [-1, 3, C3, [1024]],
    [-1, 1, SPPF, [1024, 5]], # 9
  ]

# YOLOv5 v6.0 head
head: [
    [-1, 1, Conv, [512, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 6], 1, Concat, [1]], # cat backbone P4
    [-1, 3, C3, [512, False]], # 13

    [-1, 1, Conv, [256, 1, 1]],
    [-1, 1, nn.Upsample, [None, 2, "nearest"]],
    [[-1, 4], 1, Concat, [1]], # cat backbone P3
    [-

## 2. 학습 시작

### 학습에 필요한 parameter 지정하기
- 우리가 사용하는 train.py 는 yoloV5 모델을 발표한 사람이 직접 작성한 코드이다.
- 우리는 1줄의 수정없이 코드를 그대로 사용하여 Fine Tunning을 진행한다.
- 다만 parameter를 이용해서 작업 방법을 알려준다.

### 학습(Training) 에 필요한 Parameter 지정하기

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

### 학습을 위해서 폴더 이동하고 시간 기록하기

In [ ]:
import time
%cd /content/yolov5/
%time

/content/yolov5
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


### train.py에 파라메타 전달하고 실행하기

In [ ]:
%%time
%cd /content/yolov5/
!python train.py --img 640 --batch 16 --epochs 50 --data ./pothole/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name pothole_result  --cache

/content/yolov5
2024-04-29 16:36:55.018193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 16:36:55.018277: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 16:36:55.020046: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=./models/custom_yolov5s.yaml, data=./pothole/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=,

In [ ]:
%pwd
%cat /

'/content/yolov5'

2. tensorboard를 이용한 확인

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

3. 학습 과정중에 vaildation data를 이용하여 스스로 검증한 결과를 이미지로 출력한 내용을 확인할 수 있습니다.

In [ ]:
# train을 실행할 때마다 폴더가 생기므로 !ls /content/yolov5/runs/train/pothole_results/를
!ls /content/yolov5/runs/train/pothole_results2/